In [1]:
import cv2
import os
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from scipy.spatial import distance

# Preprocess

In [2]:
# def image2array(folder):
#     images = []
#     for filename in os.listdir(folder):
#         img = cv2.imread(os.path.join(folder,filename))
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         img = cv2.resize(img, (224,224))
#         if img is not None:
#             images.append(np.array(img))
#     images = np.array(images)
#     images = images.reshape(images.shape[0], 224, 224, 3)
#     images = images.astype('float32')
#     images /= 255
#     return images

In [3]:
# train_data = image2array(r"C:\\ITM\\pants")
# print("Length of training dataset:",train_data.shape)
# # test_data = image2array(test_files)
# # print("Length of test dataset:",test_data.shape)

In [4]:
# from matplotlib import pyplot as plt
# plt.imshow(train_data[5], interpolation='nearest')
# plt.show()

# Pre-Trained Model

In [5]:
model_url = "https://tfhub.dev/tensorflow/efficientnet/lite0/feature-vector/2"

IMAGE_SHAPE = (224, 224)

layer = hub.KerasLayer(model_url, input_shape=IMAGE_SHAPE+(3,))
model = tf.keras.Sequential([layer])

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 1280)              3413024   
                                                                 
Total params: 3,413,024
Trainable params: 0
Non-trainable params: 3,413,024
_________________________________________________________________


In [7]:
def extract(file):
    file = Image.open(file).convert('L').resize(IMAGE_SHAPE)
    #display(file)

    file = np.stack((file,)*3, axis=-1)

    file = np.array(file)/255.0

    embedding = model.predict(file[np.newaxis, ...])
    #print(embedding)
    vgg16_feature_np = np.array(embedding)
    flattended_feature = vgg16_feature_np.flatten()

    #print(len(flattended_feature))
    #print(flattended_feature)
    #print('-----------')
    return flattended_feature

In [8]:
def extractAll(folder):
    images = []
    for filename in os.listdir(folder):
        img = extract(os.path.join(folder,filename))
        if img is not None:
            images.append(np.array(img))
    return images

In [9]:
pants1 = extract(r'C:\ITM\pants\3.png')
pants2 = extract(r'C:\ITM\pants\5.png')
skirt1 = extract(r'C:\ITM\skirt\1.png')

In [10]:
pants1.shape

(1280,)

In [11]:
images = extractAll(r"C:\\ITM\\test")
type(images)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


list

In [12]:
images = np.array(images)
type(images)

numpy.ndarray

In [15]:
images.shape[0]

6

In [17]:
metric = 'cosine'

In [33]:
x = extract(r'C:\ITM\pants\3.png')

In [ ]:
dc = distance.cdist([x], [pants1], metric)[0]
print(dc)
print("the distance between pants1 and the pants1 is {}".format(dc))

In [102]:
def findClosest(x,items):
    metric = 'cosine'
    dc_array = []
    for item in range(0,items.shape[0]) :
        dc = distance.cdist([x], [items[item]], metric)[0]
        if item is not None:
            dc = np.append(dc,item)
            print(dc.shape)
            dc_array.append(np.array(dc))
#     dc_array.sort()
#     dc_array = dc_array[dc_array[:,1].argsort(kind='mergesort')] #error
    return np.array(dc_array)

In [103]:
result = findClosest(x,images)

(2,)
(2,)
(2,)
(2,)
(2,)
(2,)


In [79]:
result = findClosest(skirt1,images)

(2,)
(2,)
(2,)
(2,)
(2,)
(2,)


array([0.08888063, 4.        ])

In [104]:
print(result)

[[0.         0.        ]
 [0.1613809  1.        ]
 [0.05200973 2.        ]
 [0.11066395 3.        ]
 [0.08888063 4.        ]
 [0.11392746 5.        ]]


In [18]:
dc = distance.cdist([pants1], [pants1], metric)[0]
print(dc)
print("the distance between pants1 and the pants1 is {}".format(dc))

[0.]
the distance between pants1 and the pants1 is [0.]


In [19]:
dc = distance.cdist([pants1], [pants2], metric)[0]
print(dc)
print("the distance between pants1 and the pants2 is {}".format(dc))

[0.05200973]
the distance between pants1 and the pants2 is [0.05200973]


In [20]:
type(dc)

numpy.ndarray

In [ ]:
dc = distance.cdist([pants1], [skirt1], metric)[0]
print(dc)
print("the distance between pants1 and the skirt1 is {}".format(dc))